In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path.home()/'github/IORSim_GUI'))
sys.path.append(str(Path.home()/'github/IORSim_GUI/IORlib'))
from numpy import where, set_printoptions
from IORlib.ECL import EGRID_file, UNRST_file
from Plotter import Plotter


In [ ]:
from pathlib import Path
from itertools import islice
import pyvista as pv

#pv.start_xvfb()
#pv.set_jupyter_backend('static')

cop = Path('/home/shared/owner/CoP_files')
#root = cop/'input/CF6_IORSIM_AH/CF6_IORSIM'
#root = cop/'output/IORSIM_BASECASE_1971TO2022/IORSIM_BASECASE_1971TO2022'
root = cop/'input/IORSIM_BASECASE_SURF_RES/IORSIM_BASECASE_INPUT'
#runs = Path.home()/'ekofisk/runs'
#root = runs/'K-9A_PROD/K-9A_PROD'


# egrid = EGRID_file(root)
# #dim = egrid.nijk()
# #print(dim)
i, j, k = (0, 10), (0, 10), (0,23)
# #mask = [slice(*i), slice(*j), slice(*k)]
limit = None #(3.5, 10)
# var = 'CL' #'SALT'
# grid = egrid.grid(scale=(1, 1, -1))
# unrst = UNRST_file(root)
# data = next(islice(unrst.cellarray(var), 100, None, 100))
# print(data.days)
# grid[var] = getattr(data, var).flatten()
# plotter = pv.Plotter(off_screen=False, notebook=False)
# plotter.add_mesh(grid, scalars=var, cmap='viridis', opacity='linear', show_edges=True, clim=limit)
# plotter.show() #jupyter_backend='static')
# #grid.plot()

var = 'CL'
pl = Plotter(root, wells=False, edges=False, size=(1500, 1200), culling='front')
pl.set_grid_values(res_opacity='linear', well_opacity='linear')
pl.plot(var, (i,j,k), step=1, limit=limit)

#unrst = UNRST_file(root)
#for data in unrst.cellarray('SALT'):
#    print(data.SALT[10, 10, 10])

SystemError: ERROR IORSIM_BASECASE_INPUT.UNRST not found because folder /home/shared/owner/CoP_files/input/IORSIM_BASECASE_SURF_RES is missing

In [4]:
import sys
from pathlib import Path
sys.path.append(str(Path.home()/'github/IORSim_GUI'))
sys.path.append(str(Path.home()/'github/IORSim_GUI/IORlib'))
from IORlib.ECL import Plotter

runs = Path.home()/'ekofisk/runs'
root = runs/'K-9A_PROD/K-9A_PROD'
#root = runs/'IORSIM_BASECASE_1971TO2022/IORSIM_BASECASE_1971TO2022'

#dim = (210, 266, 23)
#i, j, k = (65, 76), (75, 135), (0,23)
#i, j, k = (30, 170), (30, 170), (0,23)
limit = None #(3.5, 10)
var = 'SALT' # 'CL'

pl = Plotter(root, wells=False, edges=False, size=(1500, 1200), culling='front')
pl.set_grid_values(res_opacity='linear', well_opacity='linear')
pl.plot(var, step=1, limit=limit)
#pl.plot(var, (i,j,k), step=12, limit=limit)
#pl.plot(var, step=12, limit=limit)


ImportError: cannot import name 'Plotter' from 'IORlib.ECL' (/home/AD.NORCERESEARCH.NO/javi/github/IORSim_GUI/IORlib/ECL.py)

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.home()/'github/IORSim_GUI'))
sys.path.append(str(Path.home()/'github/IORSim_GUI/IORlib'))
import numpy as np
from IORlib.ECL import EGRID_file, UNRST_file, IX_input, RFT_file

root = Path.home()/'ekofisk/runs/IORSIM_BASECASE_1971TO2022/IORSIM_BASECASE_1971TO2022'


In [ ]:
from itertools import islice, pairwise
from pyvista import Plotter, Cylinder, Sphere, PolyData, Line, Label, Arrow
from time import sleep
from threading import Thread
from pyvistaqt import BackgroundPlotter
from IORlib.utils import flatten

def any_cell_in_box(cells, box):
    for cell in cells:
        if all(box[n][0] <= cell[n] < box[n][1] for n in range(3)):
            return True
    return False

scale = (1,1,5)
i, j, k = (75, 86), (85, 125), (0,23)
limit = (0,3.5)
var = 'CL'
wells = True
tube_opacity = 0.75
tube_size = 10
tube_height = 700
sphere_radius = 45
tube_radius = 35
well_grid_opacity = 0.5
label_fontsize = 12
arrow_scale = 400
start = 0*12
stop = None
step = 1

grid_box = (slice(*i), slice(*j), slice(*k))

unrst = UNRST_file(root)
#start = unrst.section(date=(2022, 8, 1))
celldata = unrst.cellarray(var, start=start, stop=stop, step=step)
data = next(celldata)

if wells:
    # Get position of well-grids
    inp = IX_input(root)
    wname, wtype = zip(*inp.wells())
    wpos = inp.wellpos(*wname)
    wells_inside = [n for n,cells in enumerate(wpos) if any_cell_in_box(cells, (i,j,k))]
    wname, wtype, wpos = zip(*((wname[n], wtype[n], wpos[n]) for n in wells_inside))
    wellpos = list(flatten(wpos))

# Define mask-arrays
dim = getattr(data, var).shape
box_mask = np.zeros(dim, dtype=np.bool)
box_mask[*grid_box] = True
well_mask = np.zeros(dim, dtype=np.bool)
if wells:
    well_mask[*zip(*wellpos)] = True
well_mask *= box_mask
res_mask = well_mask == False
res_mask *= box_mask

# Create grids
egrid = EGRID_file(root)
grid = egrid.grid(i, j, k, scale)
if wells:
    well_grid = grid.extract_cells(well_mask[*grid_box].flatten())
    well_grid[var] = getattr(data, var)[well_mask].flatten()
    well_grid.set_active_scalars(var)
reservoir_grid = grid.extract_cells(res_mask[*grid_box].flatten())

# Create tube grids
if wells:
    rft = RFT_file(root)
    active_wells = islice(rft.active_wells(), start, stop, step)
    time, act_wells = next(active_wells)
    #print(time, act_wells)
    tube_grid = {}
    labels = []
    zlim = grid.bounds[-1]
    for name, typ, pos in zip(wname, wtype, wpos):
        #print(name, typ, pos)
        cell_centers = np.mean(list(egrid.cell_corners(pos)), axis=1)*np.array(scale)
        tgrid = PolyData()
        tip = cell_centers[0].copy()
        tip[2] = zlim + tube_height
        labels.append(Label(name, position=tip + (0, 0, 0.5*arrow_scale), size=label_fontsize))
        dir = -1 if 'INJ' in typ else 1
        tgrid += Arrow(tip - (0, 0, 0.5*arrow_scale*dir), (0, 0, dir), scale=arrow_scale)
        tip[2] -= 0.6*arrow_scale
        for center in cell_centers:
            tgrid += Sphere(center=center, radius=sphere_radius)
            A = center - tip
            length = np.linalg.norm(A)
            tgrid += Cylinder(np.mean((tip, center), axis=0), A/length, radius=tube_radius, height=length)
            tip = center
        tube_grid[name] = tgrid

plotter = BackgroundPlotter()
#act = plotter.add_mesh(reservoir_grid, lighting=False, show_edges=True, scalars=var, cmap='jet', clim=limit)
reservoir_grid[var] = getattr(data, var)[res_mask].flatten()
reservoir_grid.set_active_scalars(var)
act = plotter.add_mesh(reservoir_grid, lighting=False, show_edges=True, cmap='jet', clim=limit)
act.mapper.dataset = act.mapper.dataset.flip_z()
act.mapper.dataset = act.mapper.dataset.flip_y()
#plotter.render()
#print(act.mapper.dataset)
#print(reservoir_grid[var])
#act.mapper.array_name = var
#print(act.mapper)

if wells:
    plotter.add_mesh(well_grid, lighting=False, show_edges=True, scalars=var, cmap='jet', clim=limit, opacity=well_grid_opacity)
    color = {'WATER_INJECTOR':'blue', 'GAS_INJECTOR':'green', 'PRODUCER':'red'}
    well_on = {}
    #well_off = {}
    for name, typ in zip(wname, wtype):
        well_on[name] = plotter.add_mesh(tube_grid[name], opacity=tube_opacity)
        #well_off[name] = plotter.add_mesh(tube_grid[name], color='white', opacity=tube_opacity)
        if name in act_wells:
            #well_on[name].SetVisibility(False)
            well_on[name].prop.color = color[typ]
        else:
            well_on[name].prop.color = 'white'
    for label in labels:
        plotter.add_actor(label)

#plotter.view_isometric()
plotter.view_yz()
plotter.show_axes()
text = plotter.add_text(str(data.date).split()[0], font_size=10)

def plot(text=text):
    for data, (time, act_wells) in zip(celldata, active_wells):
        if wells:
            well_grid[var] = getattr(data, var)[well_mask].flatten()
            for name, typ in zip(wname, wtype):
                if name in act_wells:
                    well_on[name].prop.color = color[typ]
                    #well_off[name].SetVisibility(False)
                else:
                    well_on[name].prop.color = 'white'
                    #well_off[name].SetVisibility(True)
        reservoir_grid[var] = getattr(data, var)[res_mask].flatten()
        text.SetVisibility(False)
        text = plotter.add_text(str(data.date).split()[0] + f' {time}', font_size=10)
        sleep(0.25)

#thread = Thread(target=plot)
#thread.start()

In [ ]:
pl.egrid.nijk()